In [0]:
from typing import Any, Generator, Optional, Sequence, Union, Annotated, Literal, List, Dict, Deque, Tuple
from pydantic import BaseModel, Field, ValidationError
from databricks.sdk import WorkspaceClient
from databricks_langchain import ChatDatabricks, UCFunctionToolkit, VectorSearchRetrieverTool, DatabricksVectorSearch
from databricks.vector_search.client import VectorSearchClient
from databricks.sdk.service import sql
from databricks.sdk.service.sql import StatementState

import mlflow
from mlflow.langchain.chat_agent_langgraph import ChatAgentState, ChatAgentToolNode
from mlflow.pyfunc import ChatAgent
from mlflow.types.agent import ChatAgentChunk, ChatAgentMessage, ChatAgentResponse, ChatContext

from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.language_models import LanguageModelLike
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage, SystemMessage, ToolMessage
from langchain_core.tools import BaseTool, Tool
from langchain_core.output_parsers import JsonOutputParser, StrOutputParser
from langchain.callbacks import LangChainTracer
from langchain_huggingface import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer

from langgraph.graph import END, StateGraph
from langgraph.graph.graph import CompiledGraph
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode

import base64, io, requests, re, json, pandas as pd, math, os, uuid, torch, numpy as np, wave, operator, time, tempfile, pytz, random, threading, traceback
from concurrent.futures import ThreadPoolExecutor, as_completed
from pypdf import PdfReader
from docx import Document
from datetime import date, datetime
from math import radians, sin, cos, sqrt, atan2
from random import uniform
from dotenv import load_dotenv

############################################
# databricks sdk client
############################################
load_dotenv()
DATABRICKS_HOST = os.getenv("DATABRICKS_HOST")
DATABRICKS_API_TOKEN = os.getenv("DATABRICKS_API_TOKEN")
w = WorkspaceClient(host=DATABRICKS_HOST, token=DATABRICKS_API_TOKEN)

def get_secret(scope: str, key: str) -> str:
    def _maybe_b64decode(s: str) -> str:
        try:
            dec = base64.b64decode(s, validate=True)
            if base64.b64encode(dec).decode("utf-8").strip("=") == s.strip("="):
                return dec.decode("utf-8")
        except Exception as e:
            print(f"failed to decode base64: {e}")
        return s
    try:
        resp = w.secrets.get_secret(scope=scope, key=key)
        return _maybe_b64decode(resp.value)
    except Exception as e:
        raise RuntimeError(f"failed to get secret '{scope}/{key}': {e}")

# カタログとスキーマ
UC_CATALOG = 'hhhd_demo_itec'
UC_SCHEMA = 'commuting_allowance'

# google api key
GOOGLE_API_KEY = get_secret("commute_agent", "GOOGLE_API_KEY")
# 駅すぱあと api key
EKISPERT_API_KEY = get_secret("commute_agent", "EKISPERT_API_KEY")

############################################
# LLM endpoint
############################################
LLM_ENDPOINT_NAME_1 = "databricks-claude-3-7-sonnet"
LLM_ENDPOINT_NAME_2 = "databricks-claude-sonnet-4"
LLM_ENDPOINT_NAME_3 = "databricks-claude-sonnet-4-5"
LLM_ENDPOINT_NAME_4 = "databricks-gpt-oss-120b"
LLM_ENDPOINT_NAME_5 = "databricks-gemini-2-5-flash"
LLM = ChatDatabricks(endpoint=LLM_ENDPOINT_NAME_2, temperature=0.0)

In [0]:
############################################
# tools
############################################
# ルート整形用関数
def parse_course_info(course: dict, get_serialize_data: bool = False) -> dict:
  """駅すぱあとAPIレスポンスを整形して、路線別定期代・概要・駅コードを出力"""
  try:
    # --- ユーティリティ ---
    def norm_list(x): 
      if isinstance(x, dict): return [x]
      return x or []
    def to_str(v, default="不明"): 
      return str(v).strip() if v not in (None, "") else default
    def to_int(v, default=0):
      try: return int(v)
      except Exception: return default

    # --- 基本データ ---
    route = course.get("Route", {}) or {}
    lines, points = norm_list(route.get("Line")), norm_list(route.get("Point"))
    prices = norm_list(course.get("Price"))
    serialize_data = course.get("SerializeData")
    distance = to_str(route.get("distance"))
    transfer = to_str(route.get("transferCount"))
    time_other = to_int(route.get("timeOther", 0))
    total_time = to_int(route.get("timeOnBoard")) + to_int(route.get("timeWalk")) + time_other

    # --- 定期代サマリ ---
    summary = {"Fare": "不明", "Teiki1": "不明", "Teiki3": "不明", "Teiki6": "不明"}
    for p in prices:
      k = p.get("kind") or p.get("Kind")
      if k in ["FareSummary", "Teiki1Summary", "Teiki3Summary", "Teiki6Summary"]:
        summary[k.replace("Summary", "")] = p.get("Oneway")

    # --- index別マッピング ---
    price_map = {}
    for p in prices:
      k = p.get("kind") or p.get("Kind")
      if not k or "Teiki" not in k:
        continue
      idx = str(p.get("index") or "")
      price_map.setdefault(idx, {})[k] = p.get("Oneway")

    # --- 区間情報とsectionIndex取得 ---
    route_parts, line_infos, section_index = [], [], 0
    for i, line in enumerate(lines):
      name = to_str(line.get("Name") or line.get("TypicalName"), "徒歩")
      ltype = line.get("Type")
      minutes = to_int(line.get("timeOnBoard"))
      arr_name = to_str(
        points[i + 1]["Station"]["Name"]
        if i + 1 < len(points) and isinstance(points[i + 1].get("Station"), dict)
        else points[i + 1].get("Name", "不明")
      )

      # 経路概要文
      route_parts.append(f"{name}{minutes}分→{arr_name}")

      # sectionIndexの決定（最初の鉄道・バス・船など）
      if not section_index and ltype in ["train", "bus", "ship", "plane"]:
        section_index = i + 1

      # 定期対象以外はスキップ
      if ltype in ["walk", None]:
        continue
      idx1, idx3, idx6 = map(str, [line.get("teiki1Index"), line.get("teiki3Index"), line.get("teiki6Index")])
      line_infos.append({
        "路線名": name,
        "定期代": f"1ヶ月:{price_map.get(idx1, {}).get('Teiki1', '不明')} / "
                f"3ヶ月:{price_map.get(idx3, {}).get('Teiki3', '不明')} / "
                f"6ヶ月:{price_map.get(idx6, {}).get('Teiki6', '不明')}"
      })

    # --- 駅情報の抽出（駅コードなど） ---
    station_list = []
    for p in points:
      try:
        st = p.get("Station")
        if not isinstance(st, dict):
          continue  # 駅でない（住所など）はスキップ
        geo = p.get("GeoPoint", {})
        station_list.append(f'{to_str(st.get("Name"))} {to_str(st.get("code"))}')
      except Exception as e:
        print(f"[WARN] 駅情報解析失敗: {e}")
        continue

    # --- 出力 ---
    route_infos = " → ".join(route_parts)
    result = {
      "経路概要": f"{route_infos} 列車待ち合わせ時間{time_other}分" if time_other > 0 else route_infos,
      "乗換回数": transfer,
      "平均所要時間": total_time,
      "運賃": summary["Fare"],
      "定期代": f"1ヶ月:{summary['Teiki1']} / 3ヶ月:{summary['Teiki3']} / 6ヶ月:{summary['Teiki6']}",
      "路線別定期代": line_infos,
      "駅一覧": station_list,
    }

    if get_serialize_data:
      result["serialize_data"] = serialize_data
      result["section_index"] = str(section_index or 0)

    return result

  except Exception as e:
    import traceback
    traceback.print_exc()
    return {"error": f"ルート整形時に例外が発生しました: {e}"}

# 経路検索
def search_commute_routes(requests_data: str) -> str:
  try:
    data = json.loads(requests_data)
  except Exception as e:
    return json.dumps({"error": f"JSONのパースに失敗しました: {e}"}, ensure_ascii=False)

  from_info = data.get("from_info")
  to_info = data.get("to_info")

  # 都道府県チェック（より寛容なパターン）
  pref_pattern = r"^(北海道|東京都|大阪府|京都府|.+?(都|道|府|県))"
  if not from_info or not to_info:
    return json.dumps({"error": "エラー: 出発地と到着地の住所は必須です。"}, ensure_ascii=False)
  if not re.match(pref_pattern, from_info):
    return json.dumps({"error": "from_info は都道府県から始めてください。"}, ensure_ascii=False)
  if not re.match(pref_pattern, to_info):
    return json.dumps({"error": "to_info は都道府県から始めてください。"}, ensure_ascii=False)

  date = data.get("date", None)
  date_time = data.get("time", None)

  radius = 1000  # 半径

  def fetch_one(params: dict, base_url: str, label: str):
    try:
      time.sleep(random.uniform(0, 0.15))  # 軽いジッタ
      resp = requests.get(base_url, params=params, timeout=12)
      if resp.status_code != 200:
        return label, f"API error: {resp.status_code}", []
      data_json = resp.json()
      courses = (data_json.get("ResultSet") or {}).get("Course")
      if not courses:
        return label, None, []
      # 単一要素時 dict → list に正規化
      if isinstance(courses, dict):
        courses = [courses]
      if base_url == "https://api.ekispert.jp/v1/json/search/course":
        courses = courses[:2]
      return label, None, courses
    except Exception as e:
      return label, f"API error: {e}", []

  base_url = "https://api.ekispert.jp/v1/json/search/course"
  extreme_url = "https://api.ekispert.jp/v1/json/search/course/extreme"

  base_params = {
    "key": EKISPERT_API_KEY,
    "from": from_info,
    "to": to_info,
    "searchType": "plain",
    "bus": "false"
  }
  extreme_params = {
    "key": EKISPERT_API_KEY,
    "answerCount": "20",
    "searchCount": "20",
    "viaList": f"{from_info},{radius}:{to_info},{radius}",
    "searchType": "plain",
    "sort": "time"
  }
  if date:
    base_params["date"] = date
    extreme_params["date"] = date
  if date_time:
    base_params["time"] = date_time
    base_params["searchType"] = "arrival"
    extreme_params["time"] = date_time
    extreme_params["searchType"] = "arrival"

  result_routes = {}  # dictに修正
  errors = []

  # 並列実行：ex.submit を使って Future を作る
  with ThreadPoolExecutor(max_workers=2) as ex:
    f1 = ex.submit(fetch_one, extreme_params, extreme_url, "駅すぱあとルート検索結果")
    f2 = ex.submit(fetch_one, base_params, base_url, "鉄道のみルート検索結果")
    for fut in as_completed([f1, f2]):
      label, err, courses = fut.result()
      if err:
        errors.append({label: err})
        continue
      result_routes[label] = courses

  # ルートなし
  if not result_routes:
    return json.dumps({"message": "ルートが見つかりませんでした。", "errors": errors}, ensure_ascii=False)

  # 結果データ構築
  route_datas = []
  for label, courses in result_routes.items():
    for course in courses:
      try:
        route_info = parse_course_info(course, True)
        if date_time: 
          # 前後の列車ダイヤを取得する用
          params = {
            "key": EKISPERT_API_KEY,
            "serializeData": route_info.get("serialize_data"),
            "sectionIndex": route_info.get("section_index"),
            "answerCount": "3"
          }
          route_info["schedule_params"] = params
        route_info.pop("serialize_data", None)
        route_info.pop("section_index", None)
        route_datas.append(route_info)
      except Exception as e:
        print(f"Error parsing course: {e}")
        continue

  if date_time == None:
    return json.dumps(route_datas, ensure_ascii=False)

  # 平均所要時間を取得する処理
  def get_average_time(data):
    url = "https://api.ekispert.jp/v1/json/search/course/pattern"
    try:
      # --- 共通ユーティリティ ---
      def safe_int_convert(value, default=0):
        if value in (None, "不明"): return default
        try:
          return int(value)
        except:
          return default

      params = data.get("schedule_params")

      response = requests.get(url, params=params, timeout=10)

      # --- APIが失敗した場合（HTTPエラー） ---
      if response.status_code != 200:
        print(f"get_average_time params: {params}")
        print(f"get_average_time response: {response.text}")
        data.pop("schedule_params", None)
        return data

      # --- レスポンス解析 ---
      result = response.json()
      courses = result.get("ResultSet", {}).get("Course")
      if not courses:
        print("get_average_time 検索結果がありません")
        data.pop("schedule_params", None)
        return data

      # --- 平均所要時間算出 ---
      count = 0
      total = 0
      for course in courses:
        route = course.get("Route", {}) or {}
        time_on_board = safe_int_convert(route.get("timeOnBoard"))
        time_walk = safe_int_convert(route.get("timeWalk"))
        time_other = safe_int_convert(route.get("timeOther"))
        total += time_on_board + time_walk + time_other
        count += 1
      print(f"get_average_time total: {total} count: {count}")
      data["平均所要時間"] = round(total / count, 2)  # 小数2桁で丸め
      data.pop("schedule_params", None)
      return data

    except Exception as e:
      print(f"Error getting average time: {e}")
      data.pop("schedule_params", None)
      return data

  response_datas = []
  # 並列実行：ex.submit を使って Future を作る
  with ThreadPoolExecutor(max_workers=20) as ex:
    futures = [ex.submit(get_average_time, data) for data in route_datas]
    for fut in as_completed(futures):
      response_datas.append(fut.result()) 

  return json.dumps(response_datas, ensure_ascii=False)

# 駅コードでルート取得
def get_commute_route_by_code(requests_data: str, get_serialize: bool = False) -> str:
  try:
    data = json.loads(requests_data)
  except Exception as e:
    return json.dumps({"error": f"JSONのパースに失敗しました: {e}"}, ensure_ascii=False)

  from_station_code = data.get("from_station_code")
  to_station_code = data.get("to_station_code")
  transit_station_code = data.get("transit_station_code", None)
  date = data.get("date")
  date_time = data.get("time")

  def fetch_one(params: dict, base_url: str):
    try:
      resp = requests.get(base_url, params=params, timeout=12)
      if resp.status_code != 200:
        return f"API error: {resp.status_code}"
      data_json = resp.json()
      courses = (data_json.get("ResultSet") or {}).get("Course")
      if not courses:
        return "検索結果がありません"
      # 単一要素時 dict → list に正規化
      if isinstance(courses, dict):
        courses = [courses]
      return courses[:3]
    except Exception as e:
      print(f"API error: {e}")
      return None

  extreme_url = "https://api.ekispert.jp/v1/json/search/course/extreme"
  extreme_params = {
    "key": EKISPERT_API_KEY,
    "viaList": f"{from_station_code}:{transit_station_code}:{to_station_code}" if transit_station_code else f"{from_station_code}:{to_station_code}",
    "answerCount": "20",
    "searchType": "plain",
    "sort": "time"
  }
  if date:
    extreme_params["date"] = date
  if date_time:
    extreme_params["time"] = date_time
    extreme_params["searchType"] = "arrival"

  result_routes = fetch_one(extreme_params, extreme_url)

  # ルートなし
  if not result_routes:
    return json.dumps({"message": "ルートが見つかりませんでした。"}, ensure_ascii=False)

  # 結果データ構築：label ごとに courses を展開
  response_datas = []
  for course in result_routes:
    try:
      route_info = parse_course_info(course, get_serialize)
      response_datas.append(route_info)
    except Exception as e:
      print(f"Error parsing course: {e}")
      continue
  return json.dumps(response_datas, ensure_ascii=False)

# serialize取得
def get_commute_route_serialize(requests_data: str) -> str:
  return get_commute_route_by_code(requests_data, True)

In [0]:
# 緯度経度から距離を計算する関数
def haversine(lat1, lon1, lat2, lon2):
  R = 6371  # 地球半径 (km)
  d_lat = radians(lat2 - lat1)
  d_lon = radians(lon2 - lon1)
  a = sin(d_lat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(d_lon/2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))
  return R * c  # 距離（km）

# 住所特定
def resolve_place_name_to_info(requests_data: str) -> str:
 try:
  data = json.loads(requests_data)
  place_name = data.get("place_name")
  if not place_name:
   return json.dumps({"error": "place_name は必須です。"})

  url = "https://maps.googleapis.com/maps/api/geocode/json"
  params = {"address": place_name, "key": GOOGLE_API_KEY}
  response = requests.get(url, params=params)
  result = response.json()

  if result["status"] != "OK":
   return json.dumps({"error": f"Geocoding APIエラー: {result['status']}"})

  address = result["results"][0]["formatted_address"]
  location = result["results"][0]["geometry"]["location"]

  return json.dumps({
    "address": address,
    "lat": location["lat"],
    "lng": location["lng"]
  }, ensure_ascii=False)

 except Exception as e:
  return f"error: {str(e)}"

# 歩行距離計算
def calculate_walking_distance(requests_data: str) -> str:
 try:
  data = json.loads(requests_data)
  origin = data.get("from_info")
  destination = data.get("to_info")
  if not origin or not destination:
   return json.dumps({"error": "from_info と to_info は必須です。"})

  url = "https://maps.googleapis.com/maps/api/distancematrix/json"
  params = {
   "origins": origin,
   "destinations": destination,
   "mode": "walking",
   "language": "ja",
   "key": GOOGLE_API_KEY
  }
  response = requests.get(url, params=params)
  result = response.json()

  if result["status"] != "OK":
   return json.dumps({"error": f"Distance Matrix エラー: {result['status']}"})

  info = result["rows"][0]["elements"][0]
  if info["status"] != "OK":
   return json.dumps({"error": f"ルートエラー: {info['status']}"})

  return json.dumps({
   "distance": info["distance"]["text"],
   "duration": info["duration"]["text"]
  }, ensure_ascii=False)

 except Exception as e:
  return json.dumps({"error": str(e)})

# 最寄り駅検索（駅すぱあとAPI）
def find_nearest_station_from_info(requests_data: str) -> str:
 try:
  data = json.loads(requests_data)
  address = data.get("address")
  radius = data.get("radius", 1000)
  if not address:
   return json.dumps({"error": "address は必須です。"})

  base_url = "https://api.ekispert.jp/v1/json/address/station"
  types = {
      "train": "鉄道",
      "bus": "バス",
  }

  result_stations = []
  for key, label in types.items():
    params = {
      "address": f"{address},{radius}",
      "key": EKISPERT_API_KEY,
      "type": key,
    }

    response = requests.get(base_url, params=params)
    if response.status_code != 200:
      return json.dumps({"error": f"APIエラー: {response.status_code}"})

    result = response.json()
    result_set = result.get("ResultSet", {})

    if not result_set:
      return "検索結果がありません"

    points = result_set.get("Point", [])
    if isinstance(points, dict):
      points = [points]

    station_list = points[:5]
    for point in station_list:
      station_info = point.get("Station", {})
      geo = point.get("GeoPoint", {})
      station = {
        "station_code": station_info.get("code"),
        "station_name": station_info.get("Name"),
        "yomi": station_info.get("Yomi"),
        "type": station_info.get("Type"),
        "lat": geo.get("lati_d"),
        "lng": geo.get("longi_d")
      }
      result_stations.append(station)

  return json.dumps(result_stations, ensure_ascii=False)
 except Exception as e:
  return json.dumps({"error": str(e)})

# 最寄り駅検索（Google API）
# def find_nearest_station_from_info(requests_data: str) -> str:
#   try:
#     data = json.loads(requests_data)
#     address = data.get("address")
#     radius = data.get("radius", 1000)
#     if not address:
#       return json.dumps({"error": "address は必須です。"})

#     # 住所→緯度経度
#     geo_url = f"https://maps.googleapis.com/maps/api/geocode/json?address={address}&key={GOOGLE_API_KEY}"
#     geo_resp = requests.get(geo_url).json()
#     if geo_resp["status"] != "OK":
#       return json.dumps({"error": "住所から座標を取得できません。", "status": geo_resp["status"]})
#     location = geo_resp["results"][0]["geometry"]["location"]
#     lat, lng = location["lat"], location["lng"]

#     # 駅・バス停をそれぞれ検索
#     types = ["train_station", "bus_station"]
#     all_results = {}

#     for t in types:
#       url = (
#         "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
#         f"?location={lat},{lng}&radius={radius}&type={t}&key={GOOGLE_API_KEY}"
#       )
#       resp = requests.get(url).json()

#       if resp["status"] == "OK":
#         points = []
#         for r in resp["results"]:
#           point_lat = r["geometry"]["location"]["lat"]
#           point_lng = r["geometry"]["location"]["lng"]
#           dist = haversine(lat, lng, point_lat, point_lng)
#           points.append({
#             "name": r["name"],
#             "type": t,
#             "address": r.get("vicinity"),
#             "lat": point_lat,
#             "lng": point_lng,
#             "distance_km": round(dist, 1)
#           })

#         # 距離でソートして上位3件のみ保持
#         all_results[t] = sorted(points, key=lambda x: x["distance_km"])[:3]
#       else:
#         all_results[t] = []

#     # 結果をまとめる
#     combined = []
#     for t in types:
#       combined.extend(all_results[t])

#     # 合計6件以内に制限
#     combined = sorted(combined, key=lambda x: x["distance_km"])[:6]

#     result = {
#       "input_address": address,
#       "search_radius_m": radius,
#       "count": len(combined),
#       "results": combined
#     }

#     return json.dumps(result, ensure_ascii=False, indent=2)

#   except Exception as e:
#     return json.dumps({"error": str(e)})

# 払い戻し金計算
def calculate_commute_ticket_refund(requests_data: str) -> str:
    try:
        result = json.loads(get_commute_route_serialize(requests_data))
        # resultがリストではない場合、またlenが0の場合
        if not isinstance(result, list) or len(result) == 0:
            return json.dumps({"error": "serialize_dataが取得できませんでした。"})
        serialize_data = result[0].get("serialize_data")
        data = json.loads(requests_data)
        change_section = str(data.get("change_section", "false")).lower()
        repayment_date = str(data.get("repayment_date")).strip()
        start_date = str(data.get("date")).strip() if data.get("date") else None

        if not serialize_data or not repayment_date:
            return json.dumps({"error": "serialize_data と repayment_date は必須です。"})
        
        if change_section not in ["true", "false"]:
            change_section = "false"

        url = "https://api.ekispert.jp/v1/json/course/repayment"
        params = {
            "key": EKISPERT_API_KEY,
            "serializeData": serialize_data,
            "repaymentDate": repayment_date,
            "changeSection": change_section
        }
        if start_date:
            params["startDate"] = start_date

        response = requests.get(url, params=params)
        if int(response.status_code) != 200:
            return json.dumps({"error": f"APIエラー: {response.status_code}　{response.text}"})

        def parse_repayment_response(response_text: str) -> str:
          try:
            data = json.loads(response_text)
            rs = data.get("ResultSet", {})
            repayment = rs.get("RepaymentList", {}).get("RepaymentTicket", {})

            # TeikiRouteSection を配列として取得
            teiki_route = rs.get("TeikiRoute", {}) or {}
            sections = teiki_route.get("TeikiRouteSection", [])
            if isinstance(sections, dict):
              sections = [sections]

            route_points = []
            for sec in sections:
              pts = sec.get("Point", [])
              if isinstance(pts, dict):
                pts = [pts]
              if isinstance(pts, list):
                route_points.extend(pts)

            # 駅名（先頭と末尾）を取得
            start_station = "不明"
            end_station = "不明"
            if route_points:
              # 先頭の Point から駅名
              first = route_points[0]
              if isinstance(first, dict) and "Station" in first and isinstance(first["Station"], dict):
                start_station = first["Station"].get("Name", "不明")

              # 末尾の Point から駅名
              last = route_points[-1]
              if isinstance(last, dict) and "Station" in last and isinstance(last["Station"], dict):
                end_station = last["Station"].get("Name", "不明")
              # 1駅しかない場合
              if len(route_points) == 1:
                end_station = start_station

            # 数値文字列の安全な int 変換
            def safe_int_convert(value, default=0):
              try:
                return int(value) if value is not None else default
              except (ValueError, TypeError):
                return default

            result = {
              "出発駅": start_station,
              "到着駅": end_station,
              "払い戻し日": rs.get("RepaymentList", {}).get("repaymentDate", ""),
              "購入日": rs.get("RepaymentList", {}).get("buyDate", ""),
              "有効期間（月）": repayment.get("validityPeriod", ""),
              "定期代金額": f"{safe_int_convert(repayment.get('payPriceValue')):,}円",
              "利用済金額": f"{safe_int_convert(repayment.get('usedPriceValue')):,}円",
              "手数料": f"{safe_int_convert(repayment.get('feePriceValue')):,}円",
              "払い戻し金額": f"{safe_int_convert(repayment.get('repayPriceValue')):,}円"
            }

            # 整形して出力
            lines = ["定期代払い戻し結果:"]
            for k, v in result.items():
              lines.append(f"{k}: {v}")
            return "\n".join(lines)

          except Exception as e:
            return f"整形中にエラーが発生しました: {str(e)}"

        return parse_repayment_response(response.text)
        
    except Exception as e:
        return json.dumps({"error": str(e)})

#　バス本数取得 
def count_buses_in_time_range(requests_data: str) -> str:
    try:
        data = json.loads(requests_data)
        from_stop = data.get("from_stop")
        to_stop = data.get("to_stop")
        date = data.get("date")
        start_time = data.get("start_time")
        end_time = data.get("end_time")

        if not from_stop or not to_stop or not start_time or not end_time:
            return json.dumps({"error": "from_stop, to_stop, start_time, end_time は必須です。"})

        params = {"key": EKISPERT_API_KEY, "from": from_stop, "to": to_stop}
        if date: params["date"] = date

        resp = requests.get("https://api.ekispert.jp/v1/json/bus/timetable", params=params)
        if resp.status_code != 200:
            return json.dumps({"error": f"APIエラー：{resp.status_code}"})

        def count_buses_in_time_range(response_json: dict, start_time_str: str, end_time_str: str) -> int:
          bus_lines = response_json.get("ResultSet", {}).get("TimeTable", {}).get("Line", [])
          if not bus_lines:
            return 0

          # 時間をdatetime.timeオブジェクトに変換
          time_format = "%H:%M"
          start_time = datetime.strptime(start_time_str, time_format).time()
          end_time = datetime.strptime(end_time_str, time_format).time()

          count = 0
          for line in bus_lines:
            departure_text = line.get("DepartureState", {}).get("Datetime", {}).get("text", "")
            try:
              dep_time = datetime.strptime(departure_text[:5], "%H:%M").time()
              if start_time <= dep_time <= end_time:
                count += 1
            except Exception as e:
              print(f"時間解析エラー: {e} ({departure_text})")
              continue
          return count

        # 本数カウント
        total = count_buses_in_time_range(resp.json(), start_time, end_time)
        return json.dumps({"バスの本数": total})
    except Exception as e:
        return json.dumps({"error": str(e)})

# 始発終電取得
def get_first_and_last_time(requests_data: str) -> str:
  try:
    data = json.loads(requests_data)
  except Exception as e:
    return json.dumps({"error": f"JSONのパースに失敗しました: {e}"}, ensure_ascii=False)

  base_url = "https://api.ekispert.jp/v1/json/search/course"
  from_info = data.get("from_station_code")
  to_info = data.get("to_station_code")
  via = data.get("transit_station_code")

  if not from_info or not to_info:
    return json.dumps({"error": "エラー: 出発地と到着地の情報は必須です。"}, ensure_ascii=False)

  last_train = data.get("last_train")
  first_train = data.get("first_train")

  if last_train:
    searchType = "lastTrain"
  elif first_train:
    searchType = "firstTrain"
  else:
    return json.dumps({"error": "エラー: 最終列車か最初の列車のどちらかを指定してください。"}, ensure_ascii=False)

  params = {
    "key": EKISPERT_API_KEY,
    "from": from_info,
    "to": to_info,
    "searchType": searchType
  }

  if via:
    params["via"] = via

  try:
    response = requests.get(base_url, params=params)
    if response.status_code != 200:
      return json.dumps({"error": f"APIエラー: {response.status_code}"}, ensure_ascii=False)

    api_data = response.json()
    courses = api_data.get("ResultSet", {}).get("Course")

    if not courses:
      return json.dumps({"error": "エラー: ルートが見つかりませんでした。"}, ensure_ascii=False)

    # 複数コース対応
    if isinstance(courses, list):
      course = courses[0]
    elif isinstance(courses, dict):
      course = courses
    else:
      return json.dumps({"error": f"APIエラー: 'Course'の形式が不正です（{type(courses)}）"}, ensure_ascii=False)

    route_data = course.get("Route")
    if isinstance(route_data, list):
      route = route_data[0]
    else:
      route = route_data or {}

    # Lineの形式を統一
    line_data = route.get("Line", [])
    if isinstance(line_data, dict):
      lines = [line_data]
    elif isinstance(line_data, list):
      lines = line_data
    else:
      lines = []

    # Pointの形式を統一
    point_data = route.get("Point", [])
    if isinstance(point_data, dict):
      points = [point_data]
    elif isinstance(point_data, list):
      points = point_data
    else:
      points = []

    def safe_int_convert(value, default=0):
      if value in (None, "不明"): return default
      try: return int(value)
      except: return default

    time_other = safe_int_convert(route.get("timeOther"))
    time_on_board = safe_int_convert(route.get("timeOnBoard"))
    time_walk = safe_int_convert(route.get("timeWalk"))

    # 経路情報
    line_info = []
    for i, line in enumerate(lines):
      dep = points[i] if i < len(points) else {}
      arr = points[i+1] if i+1 < len(points) else {}

      dep_station = dep.get("Station", {})
      arr_station = arr.get("Station", {})

      line_info.append({
        "transport_name": line.get("Name") or line.get("TypicalName"),
        "from_name": dep_station.get("Name") or dep.get("Name", "不明"),
        "from_code": dep_station.get("code"),
        "to_name": arr_station.get("Name") or arr.get("Name", "不明"),
        "to_code": arr_station.get("code"),
        "Departure_time": (line.get("DepartureState") or {}).get("Datetime", {}),
        "Arrival_time": (line.get("ArrivalState") or {}).get("Datetime", {})
      })

    result = {
      "search_type": searchType,
      "from": from_info,
      "to": to_info,
      "time_on_board": time_on_board,
      "time_other": time_other,
      "time_walk": time_walk,
      "total_time": time_on_board + time_other + time_walk,
      "route": line_info
    }

    if via:
      result["via"] = via

    return json.dumps(result, ensure_ascii=False, indent=2)

  except Exception as e:
    return json.dumps({"error": f"APIエラー: {e}"}, ensure_ascii=False)
  
# 指定したルートの前後1本の列車ダイヤを検索（駅すぱあとAPI）
def get_prev_and_next_schedules(requests_data: str) -> str:
    try:
        result = json.loads(get_commute_route_serialize(requests_data))
        # resultがリストではない場合、またlenが0の場合
        if not isinstance(result, list) or len(result) == 0:
            return json.dumps({"error": "serialize_dataが取得できませんでした。"})
        serialize_data = result[0].get("serialize_data")
        section_index = result[0].get("section_index")
    except Exception as e:
        return json.dumps({"error": f"serialize_data取得に失敗しました: {e}"}, ensure_ascii=False)

    if not serialize_data and not section_index:
        return json.dumps({"error": "serialize_dataとsection_indexは必須です。"})

    base_url = "https://api.ekispert.jp/v1/json/search/course/pattern"
    params = {
      "key": EKISPERT_API_KEY,
      "serializeData": serialize_data,
      "sectionIndex": section_index,
      "answerCount": "3"
    }

    try:
        response = requests.get(base_url, params=params)

        if int(response.status_code) != 200:
            return json.dumps({"error": f"APIエラー: {response.status_code}　{response.text}"})

        result = response.json()
        courses = result.get("ResultSet", {}).get("Course")

        if not courses:
            return json.dumps({})
          
    except Exception as e:
        return f"error: {str(e)}"

    # 必要なデータのみ抽出する
    def _parse_course_info(course: dict) -> dict:
      
        route = course.get("Route", {})
        lines = route.get("Line", [])
        points = route.get("Point", [])

        def safe_int_convert(value, default=0):
            if value in (None, "不明"): return default
            try: return int(value)
            except: return default

        time_other = safe_int_convert(route.get("timeOther"))
        time_on_board = safe_int_convert(route.get("timeOnBoard"))
        time_walk = safe_int_convert(route.get("timeWalk"))
        distance = route.get("distance", "不明")
        transfer_count = route.get("transferCount", "不明")

        result = {
            "time_on_board": time_on_board,
            "time_other": time_other,
            "time_walk": time_walk,
            "total_time": time_on_board + time_walk + time_other,
            "distance": distance,
            "transfer_count": transfer_count,
            "line_info": lines
        }

        return result

    # 結果データの構築
    response_datas = []
    for i, course in enumerate(courses):
        try:
            route_info = _parse_course_info(course)
            response_datas.append(route_info)
        except Exception as e:
            print(f"Error parsing course: {e}")
            continue

    return json.dumps(response_datas, ensure_ascii=False)

# 本日の日付と現在の時刻を取得
def get_current_datetime(requests_data: str = None) -> str:
    jst = pytz.timezone('Asia/Tokyo')
    now = datetime.now(jst)
    weekday_jp = ["月", "火", "水", "木", "金", "土", "日"]
    weekday = weekday_jp[now.weekday()]
    return now.strftime("%Y-%m-%d") + f"({weekday})" + now.strftime(" %H:%M:%S")

In [0]:
tools = [
  Tool(
    name="search_commute_routes",
    func=search_commute_routes,
    description= """
    JSON形式の文字列で渡された出発地と到着地の「住所」(駅名は不可です)を使って、駅すぱあとAPIからルート（駅名、駅コード含む）を取得するツールです。
    返り値の「平均所要時間」は、基準となる1本のダイヤに加えて、その前後の各1本を含めた合計3本の所要時間の平均として算出されています。

    注意事項:
    - "from_info" は出発地で、必ず都道府県から始まる完全な日本語**住所**（ビル名、マンション名、部屋番号などは不要）で指定してください。
    - "to_info" は到着地で、必ず都道府県から始まる完全な日本語**住所**（ビル名、マンション名、部屋番号などは不要）で指定してください。
    - "data"（任意）は検索する日付で、定期券の場合は利用開始日になります。Format: YYYYMMDD
    - "time"（任意）は到着時間で、Format: HHMM

    入力形式（JSON文字列）:
    - 基本入力: '{"from_info": "大阪府大阪市中央区1-1-1", "to_info": "大阪府大阪市福島区1-2-3"}'
    - 日付 + 時間指定: '{"from_info": "大阪府大阪市中央区1-1-1", "to_info": "大阪府大阪市福島区1-2-3", "data": "20230901", "time": "1205"}'
    """
  ),
  Tool(
    name="get_commute_route_by_code",
    func=get_commute_route_by_code,
    description= """
    JSON形式の文字列で渡された出発駅と到着駅の駅コードを使って、駅すぱあとAPIからルートを取得するツールです。

    注意事項:
    - "from_station_code" は出発駅で、駅コードで指定してください。
    - "to_station_code" は到着駅で、駅コードで指定してください。
    - "transit_station_code" （任意）は経由駅で、駅コードで指定してください。
    - "data"（任意）は検索する日付で、定期券の場合は利用開始日になります。Format: YYYYMMDD
    - "time"（任意）は到着時間で、Format: HHMM

    入力形式（JSON文字列）:
    - 基本入力: '{"from_station_code": "22828", "to_station_code": "25853"}'
    - 日付 + 時間指定: '{"from_station_code": "22828", "to_station_code": "25853", "transit_station_code": "22358", "time": "1205"}'
    """
  ),
  Tool(
    name="get_commute_route_serialize",
    func=get_commute_route_serialize,
    description= """
    JSON形式の文字列で渡された出発駅と到着駅の駅コードを使って、serializeData（経路情報）を取得するツールです。

    注意事項:
    - "from_station_code" は出発駅で、駅コードで指定してください。
    - "to_station_code" は到着駅で、駅コードで指定してください。
    - "transit_station_code" （任意）は経由駅で、駅コードで指定してください。
    - "data"（任意）は検索する日付で、定期券の場合は利用開始日になります。Format: YYYYMMDD
    - "time"（任意）は到着時間で、Format: HHMM

    入力形式（JSON文字列）:
    - 基本入力: '{"from_station_code": "22828", "to_station_code": "25853"}'
    - 日付 + 時間指定: '{"from_station_code": "22828", "to_station_code": "25853", "transit_station_code": "22358", "time": "1205"}'
    """
  ),
  Tool(
    name="resolve_place_name_to_info",
    func=resolve_place_name_to_info,
    description="""
    曖昧な地名や駅名（例: "東京駅", "吉祥寺", "新宿西口"）をAPIを使って正確な住所と緯度・経度に変換するツールです。
    住所が不明確な場合でも、一般的な地名から詳細な地理情報を取得できます。
    入力形式はJSON文字列で、"place_name"キーに検索対象の地名を指定してください。
    入力例: '{"place_name": "東京駅"}'
    """
  ),
  Tool(
    name="find_nearest_station_from_info",
    func=find_nearest_station_from_info,
    description="""
    指定した日本語住所を元に、駅すぱあと APIを利用して、周辺（半径指定可能）の鉄道駅およびバス停を検索するツールです。
    鉄道駅・バス停それぞれ最大5件ずつ（合計最大10件）を距離順に取得します。
    通勤・通学経路の起点判定や、周辺交通アクセスの自動可視化などに活用できます。

    注意事項:
    - "address"（必須）は必ず都道府県から始まる完全な日本語住所で指定してください（ビル名などは不要です）
    - "radius"（任意）は検索半径（メートル）で、例: 1500（= 半径1.5km）指定しない場合はデフォルトで半径1kmとして扱われます。

    入力形式（JSON文字列）:
    - 基本入力: '{"address": "東京都三鷹市下連雀3丁目10-12"}'
    - 鉄道駅に限定 + 半径1km指定: '{"address": "東京都武蔵野市吉祥寺南町2-1-1", "radius": 1500}'
    """
  ),
  Tool(
    name="calculate_walking_distance",
    func=calculate_walking_distance,
    description="""
    2つの住所間の徒歩による距離と所要時間をAPIを使って算出するツールです。
    入力形式はJSON文字列で、"from_info"と"to_info"にそれぞれの住所を指定してください。
    入力例: '{"from_info": "東京都三鷹市下連雀3丁目10-12", "to_info": "吉祥寺駅"}'
    """
  ),
  Tool(
    name="calculate_commute_ticket_refund",
    func=calculate_commute_ticket_refund,
    description="""
    駅すぱあとAPIを使用して、定期券の払い戻し金額を計算するツールです。

    注意事項:
    - "from_station_code" は出発駅で、駅コードで指定してください。
    - "to_station_code" は到着駅で、駅コードで指定してください。
    - "date" は定期券の利用開始日で、Format: YYYYMMDD。
    - "repayment_date" は払い戻しを希望する日付で、Format: YYYYMMDD。
    - "change_section" （任意）は区間変更かどうかを指定します。trueの場合、区間変更の払い戻し計算を行います。falseの場合、区間変更の払い戻し計算を行いません。デフォルトはfalseです。

    入力例（JSON形式）:
    - 区間変更しない: '{"from_station_code": "22828", "to_station_code": "25853", "date": "20300401", "repayment_date": "20300630"}'
    - 区間変更: '{"from_station_code": "22828", "to_station_code": "25853", "date": "20300401", "repayment_date": "20300630", "change_section": true,}'
    """
  ),
  Tool(
    name="count_buses_in_time_range",
    func=count_buses_in_time_range,
    description="""
    指定したバス停（from_stop, to_stop）間のバス時刻表から、指定した時間帯（start_time〜end_time）に運行されるバスの本数をカウントするツールです。

    【注意点】
    - "from_stop" と "to_stop" は駅すぱあとの駅名あるいは駅コードで指定してください（例: "東京駅八重洲口／都営バス","123456"）。
    - "start_time" および "end_time" は "HH:MM" 形式で指定してください（例: "07:00", "09:00"）。
    - "date" は任意で指定可能ですが、指定する場合は "YYYYMMDD" 形式で入力してください。
    - API制限により、対応しないバス停や時刻の場合は本数が0件になることがあります。

    【入力例】
    {
      "from_stop": "東京駅八重洲口／都営バス",
      "to_stop": "南千住駅西口／都営バス",
      "date": "20250701",
      "start_time": "07:00",
      "end_time": "09:00"
    }
    """
  ),
  Tool(
    name="get_first_and_last_time",
    func=get_first_and_last_time,
    description = """
    出発駅（from_station_code）と到着駅（to_station_code）を指定して、始発または終電の情報を検索するツールです。

    【注意点】
    - "from_station_code" は出発駅で、駅コードで指定してください。
    - "to_station_code" は到着駅で、駅コードで指定してください。
    - "data"（任意）は検索する日付で、定期券の場合は利用開始日になります。Format: YYYYMMDD
    - "time"（任意）は到着時間で、Format: HHMM

    【入力例】
    {
    "from_station_code": "123456",
    "to_station_code": "223456",
    "first_train": true
    }
    """
  ),
  Tool(
    name="get_prev_and_next_schedules",
    func=get_prev_and_next_schedules,
    description = """
    駅すぱあとAPIを使用して、指定した経路情報の前後1本の列車ダイヤを検索するツールです。
    
    注意事項:
    - "from_station_code" は出発駅で、駅コードで指定してください。
    - "to_station_code" は到着駅で、駅コードで指定してください。
    - "data" は検索する日付で、定期券の場合は利用開始日になります。Format: YYYYMMDD
    - "time" は到着時間で、Format: HHMM

    入力例（JSON形式）: {"from_station_code": "22828", "to_station_code": "25853", "data": "20251030", "time": "1205"}'
    """
  ),
  Tool(
      name="get_current_datetime",
      func=get_current_datetime,
      description="本日の日付（曜日）と現在の時刻を取得するツールです。（引数不要）"
  )
]

#####################
## Define agent logic
#####################
# LangGraphベースのエージェント構築
def create_tool_calling_agent(
    model: LanguageModelLike,
    tools: Union[ToolNode, Sequence[BaseTool]]
    ) -> CompiledGraph:
    # モデルにツールをバインド
    model = model.bind_tools(tools)

    # 次の状態を判断する関数
    def should_continue(state: ChatAgentState) -> str:
        last_msg = state["messages"][-1]
        tools_ran = state.get("tools_ran", set())
        if "tool_calls" in last_msg and last_msg["tool_calls"]:
            return "tools"
        return "end"

    def _maybe_b64decode(s: str) -> str:
        try:
            dec = base64.b64decode(s, validate=True)
            if base64.b64encode(dec).decode("utf-8").strip("=") == s.strip("="):
                return dec.decode("utf-8")
        except Exception as e:
            print(f"failed to decode base64: {e}")
        return s

    def get_secret(scope: str, key: str) -> str:
        try:
            load_dotenv()
            DATABRICKS_HOST = os.getenv("DATABRICKS_HOST")
            DATABRICKS_API_TOKEN = os.getenv("DATABRICKS_API_TOKEN")
            w = WorkspaceClient(host=DATABRICKS_HOST, token=DATABRICKS_API_TOKEN)
            resp = w.secrets.get_secret(scope=scope, key=key)
            return _maybe_b64decode(resp.value)
        except Exception as e:
            raise RuntimeError(f"failed to get secret '{scope}/{key}': {e}")

    # system prompt を動的に取得
    def preprocessor(state: ChatAgentState):
        system_prompt = get_secret("commute_agent", "WOKER_SYSTEM_PROMPT")
        messages = state["messages"]
        if system_prompt:
            messages = [{"role": "system", "content": system_prompt}] + messages
        return messages

    preprocessor = RunnableLambda(preprocessor)

    # パイプライン定義
    model_runnable = preprocessor | model
    
    # def call_model(state: ChatAgentState, config: RunnableConfig):
    #     response = model_runnable.invoke(state, config)
    #     if response is None:
    #         return {"messages": []}

    #     return {"messages": [response]}

    def _extract_usage(resp):
      rm = getattr(resp, "response_metadata", None)
      if isinstance(rm, dict):
        u = rm.get("usage") or rm.get("token_usage")
        if isinstance(u, dict):
          return {
            "in": u.get("prompt_tokens") or u.get("input_tokens"),
            "out": u.get("completion_tokens") or u.get("output_tokens"),
            "total": u.get("total_tokens"),
          }
        if any(k in rm for k in ("prompt_tokens", "completion_tokens", "total_tokens")):
          return {
            "in": rm.get("prompt_tokens"),
            "out": rm.get("completion_tokens"),
            "total": rm.get("total_tokens"),
          }

      # additional_kwargs 側
      ak = getattr(resp, "additional_kwargs", None)
      if isinstance(ak, dict):
        u = ak.get("usage") or ak.get("token_usage")
        if isinstance(u, dict):
          return {
            "in": u.get("prompt_tokens") or u.get("input_tokens"),
            "out": u.get("completion_tokens") or u.get("output_tokens"),
            "total": u.get("total_tokens"),
          }

      # OpenAI/Anthropic 生 dict の場合
      if isinstance(resp, dict):
        u = resp.get("usage") or resp.get("token_usage")
        if isinstance(u, dict):
          return {
            "in": u.get("prompt_tokens") or u.get("input_tokens"),
            "out": u.get("completion_tokens") or u.get("output_tokens"),
            "total": u.get("total_tokens"),
          }

      return None

    def _extract_model_meta(resp):
      rm = getattr(resp, "response_metadata", None)
      if isinstance(rm, dict):
        return {
          "model": rm.get("model") or rm.get("model_name"),
          "finish": rm.get("finish_reason"),
        }
      if isinstance(resp, dict):
        return {
          "model": resp.get("model"),
          "finish": resp.get("finish_reason"),
        }
      return {"model": None, "finish": None}

    # トークン使用状況
    def log_token_usage(response, itpm_limit: int, otpm_limit: int):
      u = _extract_usage(response)
      meta = _extract_model_meta(response)

      if not u:
        print("[TOKENS] no-usage (checked response_metadata/additional_kwargs)")
        if meta.get("model") or meta.get("finish"):
          print(f"[MODEL] model={meta.get('model')}, finish={meta.get('finish')}")
        return

      in_toks = int(u.get("in") or 0)
      out_toks = int(u.get("out") or 0)
      total_toks = int(u.get("total") or (in_toks + out_toks))

      def pct(x, limit):
        return f"{(x/limit*100):.1f}%" if limit and limit > 0 else "-"

      itpm_pct = pct(in_toks, itpm_limit)
      otpm_pct = pct(out_toks, otpm_limit)

      # 8割超で警告
      if itpm_limit and in_toks > itpm_limit * 0.8:
        print(f"[WARN] Approaching ITPM: {in_toks}/{itpm_limit}")
      if otpm_limit and out_toks > otpm_limit * 0.8:
        print(f"[WARN] Approaching OTPM: {out_toks}/{otpm_limit}")

    # レート制限エラーハンドリング
    def handle_rate_limit_error(e: Exception):
      payload = None
      try:
        resp_obj = getattr(e, "response", None)
        if resp_obj is not None:
          try:
            payload = resp_obj.json()
          except Exception:
            try:
              payload = json.loads(getattr(resp_obj, "text", "") or "{}")
            except Exception:
              payload = None
      except Exception:
        payload = None

      # OpenAI/Anthropic 由来の body を例外文字列から抽出
      if payload is None:
        s = str(e)
        try:
          start = s.find("{")
          end = s.rfind("}")
          if start != -1 and end != -1 and end > start:
            payload = json.loads(s[start:end+1])
        except Exception:
          payload = None

      if isinstance(payload, dict):
        err_code = payload.get("error_code") or payload.get("code") or "RATE_LIMIT"
        msg = payload.get("message") or payload.get("error") or "Rate limit exceeded"
        # どの制限かヒント抽出
        hint = "unknown"
        m = str(msg).lower()
        if "input token" in m or "input tokens per minute" in m:
          hint = "ITPM"
        elif "output token" in m or "output tokens per minute" in m:
          hint = "OTPM"
        elif "requests per hour" in m or "qph" in m:
          hint = "QPH"
        print(f"[ERROR] {err_code}: {msg}  (likely={hint})")
        return

      print(f"[ERROR] Rate limit exceeded: {e}")

    # レート制限判定
    def _is_rate_limit_error(e: Exception):
      # 429 ステータス優先
      try:
        resp = getattr(e, "response", None)
        if resp is not None:
          sc = getattr(resp, "status_code", None)
          if sc == 429:
            # 本文から ITPM/OTPM/QPH を推定
            try:
              body = resp.json()
            except Exception:
              body = getattr(resp, "text", "") or ""
            return True, _limit_hint_from_payload_or_text(body)
      except Exception:
        pass

      # 例外文字列にヒント
      s = str(e).lower()
      if "429" in s or "rate limit" in s or "request_limit_exceeded" in s:
        return True, _limit_hint_from_payload_or_text(s)
      return False, "unknown"

    def _limit_hint_from_payload_or_text(payload):
      text = ""
      if isinstance(payload, dict):
        text = (payload.get("message") or payload.get("error") or json.dumps(payload)).lower()
      else:
        text = str(payload).lower()
      if "input token" in text or "input tokens per minute" in text:
        return "ITPM"
      if "output token" in text or "output tokens per minute" in text:
        return "OTPM"
      if "requests per hour" in text or "qph" in text:
        return "QPH"
      return "unknown"

    # call_model（429 のみ指数バックオフ付きで再試行）
    def call_model(state: ChatAgentState, config: RunnableConfig):
      t0 = time.time()

      # リトライ設定
      cfg = (config.get("configurable") if isinstance(config, dict) else None) or {}
      max_attempts = int(cfg.get("retry_attempts", 3))           # 総試行回数（初回＋再試行）
      base_delay = float(cfg.get("retry_base_sec", 2.0))         # 初回待機の基準秒（指数倍）
      max_wait   = float(cfg.get("retry_max_wait_sec", 60.0))    # 一回の待機最大秒

      attempt = 1
      last_err = None
      while attempt <= max_attempts:
        try:
          resp = model_runnable.invoke(state, config)
          dt_ms = int((time.time() - t0) * 1000)
          # print(f"[OK] model.invoke done: {dt_ms} ms (attempt {attempt}/{max_attempts})")

          # トークン表示
          ITPM_LIMIT = 50_000
          OTPM_LIMIT = 5_000
          log_token_usage(resp, itpm_limit=ITPM_LIMIT, otpm_limit=OTPM_LIMIT)

          try:
            preview = getattr(resp, "content", None)
            if preview is None and isinstance(resp, dict):
              preview = resp.get("content")
            if preview:
              text = str(preview)
              if len(text) > 200:
                text = text[:200] + "..."
          except Exception:
            pass

          if resp is None:
            return {"messages": []}
          return {"messages": [resp]}

        except Exception as e:
          last_err = e
          is_rl, hint = _is_rate_limit_error(e)
          if not is_rl:
            # レート制限以外は即エラーを表に返す
            print(f"[ERROR] Non-rate-limit error on attempt {attempt}: {e}")
            raise

          try:
            handle_rate_limit_error(e)
          except Exception:
            print(f"[ERROR] Rate limit (likely={hint}): {e}")

          if attempt >= max_attempts:
            print(f"[ERROR] Retry exhausted ({max_attempts} attempts). Raising last error.")
            raise last_err

          # 指数バックオフ + ジッター
          delay = min(max_wait, base_delay * (2 ** (attempt - 1)))
          jitter = uniform(0.5, 1.0)
          wait_sec = delay * jitter
          # print(f"[RETRY] attempt {attempt+1}/{max_attempts} after {wait_sec:.1f}s (limit={hint})")
          time.sleep(wait_sec)
          attempt += 1

      if last_err:
        raise last_err
      return {"messages": []}

    # 新しい状態stateを定義
    workflow = StateGraph(ChatAgentState)
    tool_node = ChatAgentToolNode(tools)
    # nodeを追加
    workflow.add_node("agent", RunnableLambda(call_model))
    workflow.add_node("tools", tool_node)

    ###########################
    ## 状態遷移ルールを定義
    ## agent → "tool_calls"あり → tools
    ## tools → 呼び出し後 → agent に戻る（再応答）
    ###########################
    # 入口
    workflow.set_entry_point("agent")
    # 条件edge追加
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            "end": END,
        },
    )
    # 一般edge追加
    workflow.add_edge("tools", "agent")
    # コンパイル
    return workflow.compile()

# 出力結果の形を整形する
class LangGraphChatAgent(ChatAgent):
    def __init__(self, agent: CompiledStateGraph, experiment_id: Optional[str] = None):
      self.agent = agent

    def predict(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> ChatAgentResponse:
        request = {"messages": self._convert_messages_to_dict(messages)}
        config = {"recursion_limit": 60}
        final_messages = []
        for event in self.agent.stream(request, config=config, stream_mode="updates"):
            for node_data in event.values():
                final_messages = [
                    ChatAgentMessage(**msg) for msg in node_data.get("messages", [])
                ]
        return ChatAgentResponse(messages=final_messages)

    def predict_stream(
        self,
        messages: list[ChatAgentMessage],
        context: Optional[ChatContext] = None,
        custom_inputs: Optional[dict[str, Any]] = None,
    ) -> Generator[ChatAgentChunk, None, None]:
        request = {"messages": self._convert_messages_to_dict(messages)}
        config = {"recursion_limit": 60}
        for event in self.agent.stream(request, config=config, stream_mode="updates"):
            for node_data in event.values():
                yield from (
                    ChatAgentChunk(**{"delta": msg}) for msg in node_data["messages"]
                )

agent = create_tool_calling_agent(LLM, tools)
AGENT = LangGraphChatAgent(agent)
mlflow.models.set_model(AGENT)